In [7]:
import numpy as np
import random as rm

# THEMES
Add different themes for the system to use, which will control the ranges and values that the musical and timbral parameters can take  
eg. lush, drone, busy, minimal, etc.  



- minimal- One note sustained  
- drone- long sustained chords, sawtooth wave heavy
- lush- full sounding, minimal effects and minimal chord selection
- busy- lots of chords, lots of effects, varying sustain for each note
- voice/choir synth- few chords, not as long sustained as drone/minimal

## Testing

In [2]:
# The statespace
states = ["Sleep","Icecream","Run"]

# Possible sequences of events
transitionName = [["SS","SR","SI"],["RS","RR","RI"],["IS","IR","II"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.2,0.6,0.2],[0.1,0.6,0.3],[0.2,0.7,0.1]]

In [3]:
if sum(transitionMatrix[0])+sum(transitionMatrix[1])+sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: print("All is gonna be okay, you should move on!! ;)")


All is gonna be okay, you should move on!! ;)


In [4]:
# A function that implements the Markov model to forecast the state/mood.
def activity_forecast(days):
    # Choose the starting state
    activityToday = "Sleep"
    print("Start state: " + activityToday)
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    activityList = [activityToday]
    i = 0
    # To calculate the probability of the activityList
    prob = 1
    while i != days:
        if activityToday == "Sleep":
            change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
            if change == "SS":
                prob = prob * 0.2
                activityList.append("Sleep")
                pass
            elif change == "SR":
                prob = prob * 0.6
                activityToday = "Run"
                activityList.append("Run")
            else:
                prob = prob * 0.2
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Run":
            change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
            if change == "RR":
                prob = prob * 0.5
                activityList.append("Run")
                pass
            elif change == "RS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.3
                activityToday = "Icecream"
                activityList.append("Icecream")
        elif activityToday == "Icecream":
            change = np.random.choice(transitionName[2],replace=True,p=transitionMatrix[2])
            if change == "II":
                prob = prob * 0.1
                activityList.append("Icecream")
                pass
            elif change == "IS":
                prob = prob * 0.2
                activityToday = "Sleep"
                activityList.append("Sleep")
            else:
                prob = prob * 0.7
                activityToday = "Run"
                activityList.append("Run")
        i += 1  
    print("Possible states: " + str(activityList))
    print("End state after "+ str(days) + " days: " + activityToday)
    print("Probability of the possible sequence of states: " + str(prob))

# Function that forecasts the possible state for the next 2 days
activity_forecast(2)


Start state: Sleep
Possible states: ['Sleep', 'Run', 'Run']
End state after 2 days: Run
Probability of the possible sequence of states: 0.3


# Note Selection

In [9]:
import numpy as np
import pandas as pd
from collections import Counter
np.random.seed(42)

# read file
data = pd.read_csv('../Liverpool_band_chord_sequence.csv')
# ----------------------------------------------------------------------
n = 3
chords = data['chords'].values
ngrams = zip(*[chords[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

# bigrams[:5]
# ----------------------------------------------------------------------
def predict_next_state(chord:str, data:list=bigrams, segment:int=1):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # print(options)
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

# ----------------------------------------------------------------------
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    chords.append(chord)
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords


In [10]:
extended_chord_sequence = generate_sequence('C')

# Note duration

- Changes in note durations
- minumum note duration is 0.5s
- maximum note duration is 8 seconds
- To be modeled using a markov chain

In [11]:
# note_durations = [0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
# The statespace
states = ["VS","S","M","L","VL"]
# Possible sequences of events
transitionName = [["VSVS","VSS","VSM","VSL","VSVL"],["SVS","SS","SM","SL","SVL"],["MVS","MS","MM","ML","MVL"],["LVS","LS","LM","LL","LVL"],["VLVS","VLS","VLM","VLL","VLVL"]]

# Probabilities matrix (transition matrix)
#TODO: Load transition matrix, last_note_min_length from each theme
transitionMatrix = np.array([
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2],
                             [0.2,0.2,0.2,0.2,0.2]
                             ])
assert sum(transitionMatrix[0])==1
assert sum(transitionMatrix[1])==1
assert sum(transitionMatrix[2])==1
assert sum(transitionMatrix[3])==1
assert sum(transitionMatrix[4])==1


def get_length_sequence(initial_state, n_steps, seed="NA"):
    sequence = [initial_state]
    if seed != "NA":
        np.random.seed(seed)
    for i in range(n_steps-1):
        transition_probs = transitionMatrix[states.index(sequence[-1])]
        next_state = np.random.choice(states, p=transition_probs)
        sequence.append(next_state)
    return sequence

def get_durations(total_length, sequence, last_note_min_length=3):
    length_reqd = total_length
    durations = []
    for i in range(len(sequence)):
        if sum(durations) > total_length:
            # print(durations)
            durations = durations[:-1]
            durations.append(round(total_length-sum(durations),2))
            # print(durations)
            # print(sum(durations))
            while durations[-1] < last_note_min_length:
                durations[-2] += durations[-1]
                durations=durations[:-1]
                # print(durations)
                # print(sum(durations))
            # print("exited while")
            # print(durations)
            # print(sum(durations))
            assert durations[-1] >= last_note_min_length
            assert round(sum(durations),2) == total_length
            # l = len(durations)
            return durations
            
        if i == len(sequence)-1:
            if length_reqd < last_note_min_length:
                durations.append(length_reqd)
                break
        if sequence[i] == "VS":
            durations.append(round(rm.uniform(0.5,1),2))
        elif sequence[i] == "S":
            durations.append(round(rm.uniform(1,2),2))
        elif sequence[i] == "M":
            durations.append(round(rm.uniform(2,3),2))
        elif sequence[i] == "L":
            durations.append(round(rm.uniform(3,5),2))
        elif sequence[i] == "VL":
            durations.append(round(rm.uniform(5,8),2))
        length_reqd -= durations[i]
    l = len(durations)
    assert durations[-1] >= last_note_min_length
    assert round(sum(durations),2) == total_length
    return durations

In [12]:
lengths = get_length_sequence("VL",5)
chord_sequence = extended_chord_sequence[:len(lengths)]
lengths

['VL', 'L', 'VS', 'VS', 'VL']

In [13]:
durs = get_durations(20,get_length_sequence("VL",20),2)
print(durs)
print(sum(durs))

[5.98, 7.47, 6.55]
20.0


# Note Density
- Number of simultaneous notes  
- Ranges from 1 to 5  
- Modeled using a markov chain  


Load transition matrix for note density based on theme

In [14]:
def get_note_density(sequence, initial_state=3, seed="NA", transition_probs=[0.2,0.3,0.3,0.1,0.1]):
    assert sum(transition_probs) == 1
    states = [1,2,3,4,5]
    densities=[initial_state]
    for i in range(len(sequence)-1):
        densities.append(np.random.choice(states, p=transition_probs))
    return densities

In [17]:
get_note_density(extended_chord_sequence)

[3,
 4,
 3,
 5,
 1,
 1,
 1,
 2,
 2,
 2,
 4,
 2,
 2,
 3,
 1,
 4,
 1,
 5,
 3,
 1,
 1,
 4,
 3,
 3,
 3,
 1,
 2,
 1,
 4,
 3,
 2]

# Melodic Range
- Number of octaves to select from (Range: 1 - 3)
- Specific octave to use (Changes from one timbre to another)
- Modeled using random sampling with probabilities

In [18]:
def get_octaves(sequence, initial_state=3, seed="NA", transition_probs=[0.3,0.5,0.1,0.1]):
    assert sum(transition_probs) == 1
    octaves=[initial_state]
    for i in range(len(sequence)-1):
        octaves.append(np.random.choice([2,3,4,5], p=transition_probs))
    return octaves

In [19]:
get_octaves(chord_sequence)

[3, 2, 3, 3, 3]

# Loudness
- Scaling factor for the final output (Range: 0 - 1)
- Initial and final values to be selected and connected using a Gensound Linear or Exponential object

# Melodic Direction
- Ascending or descending based on the selected segment

# Combined params

In [20]:
def get_chord_and_metadata(total_length, ending_chord_length,starting_chord='C', starting_length="VL", starting_density=1, starting_octave=3, seed="NA"):
    chord_sequence = generate_sequence('C')
    lengths = get_length_sequence(starting_length, 5, seed)
    durations = get_durations(total_length,lengths,ending_chord_length)
    chord_sequence = chord_sequence[:len(durations)]

    densities = get_note_density(chord_sequence, starting_density, seed)
    octaves = get_octaves(chord_sequence)

    assert durations[-1] >= ending_chord_length
    return {'chords':chord_sequence, 'durations': durations, 'note_densities':densities, 'octaves':octaves}

In [21]:
get_chord_and_metadata(20,2)

AssertionError: 